In [ ]:
import { createAgent, Calculator, DynamicTool, InMemoryStore } from "npm:langchain";
import { ChatDeepSeek } from "npm:@langchain/deepseek";
import { ChatOpenAI } from "@langchain/openai";
import { config } from "npm:dotenv";
config();

In [29]:
// 1. 定义天气查询工具
const getWeather = async (city: string): Promise<string> => {
  const apiKey = process.env.WEATHER_API_KEY;
  const url = `http://api.weatherapi.com/v1/current.json?key=${apiKey}&q=${city}`;
  const response = await fetch(url);
  const data = await response.json();
  const temp = data.current.temp_c;
  const condition = data.current.condition.text;
  return `${city}的温度是${temp}°C，天气${condition}`;
};

In [30]:
// 2. 配置工具
const tools = [
  new DynamicTool({
    name: "Weather",
    description: "当用户询问天气时使用。输入应该是城市名称。",
    func: getWeather
  })
];

In [31]:
// 3. 配置 LLM：使用 OpenAI 的 ChatOpenAI 作为 agent 的模型（DeepSeek 可用于检索/工具）
const model = new ChatOpenAI({
    apiKey: process.env.OPENAI_API_KEY,
    modelName: "deepseek-chat", 
    configuration: {
        baseURL: "https://api.deepseek.com/v1",
    },
    temperature: 0.1,
    model: "gpt-3.5-turbo",
});

In [32]:
// 4. 配置记忆（自动管理对话历史）
const memory = new InMemoryStore({
  memoryKey: "chat_history",
  returnMessages: true
});

In [33]:
// 5. 创建 Agent
const executor = await createAgent(
  model,
  tools,
  {
    agentType: "structured-chat-zero-shot-react-description",
    memory: memory,
    verbose: true,
  }
);

In [34]:
// 6. 使用
const response = await executor.invoke({
  input: "上海今天天气怎么样？"
});
console.log(response.output);

: 

In [ ]:
// 7. 继续对话（自动记住上下文）
const response2 = await executor.invoke({
  input: "那北京呢？"
});
console.log(response2.output);